In [1]:
!pip install pypdf2
!pip install PyMuPDF

  Obtaining dependency information for PyMuPDFb==1.23.9 from https://files.pythonhosted.org/packages/13/e9/bbe35512ed88b909d39140c3f554d6d1bbb704ccd6e5f25f4e54ba747e01/PyMuPDFb-1.23.9-py3-none-win_amd64.whl.metadata
  Using cached PyMuPDFb-1.23.9-py3-none-win_amd64.whl.metadata (1.3 kB)
Using cached PyMuPDFb-1.23.9-py3-none-win_amd64.whl (24.5 MB)
  Attempting uninstall: PyMuPDFb
    Found existing installation: PyMuPDFb 1.23.22
    Uninstalling PyMuPDFb-1.23.22:
      Successfully uninstalled PyMuPDFb-1.23.22


In [2]:
import tkinter as tk
from tkinter import filedialog, simpledialog
import fitz  # PyMuPDF
import glob
import os
import shutil

def find_keyword_end_page(pdf_path, keyword):
    doc = fitz.open(pdf_path)
    found_page = None  # 키워드 발견된 페이지 번호

    for i, page in enumerate(doc):
        text = page.get_text()
        if keyword in text:
            found_page = i
            break
            
    doc.close()
    return found_page

def extract_pages_to_new_pdf(pdf_path, end_page, output_folder, output_suffix):
    doc = fitz.open(pdf_path)
    new_doc = fitz.open()
    for page_num in range(end_page):
        new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
    fname = os.path.splitext(os.path.basename(pdf_path))[0]
    output_path = os.path.join(output_folder, f'{fname}_{output_suffix}.pdf')
    new_doc.save(output_path)
    new_doc.close()
    doc.close()
    print(f"Extracted pages up to {end_page} from {pdf_path} to {output_path}")

def extract_remaining_pages_to_new_pdf(pdf_path, start_page, etc_folder, output_suffix):
    doc = fitz.open(pdf_path)
    new_doc = fitz.open()
    for page_num in range(start_page, doc.page_count):
        new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
    fname = os.path.splitext(os.path.basename(pdf_path))[0]
    output_path = os.path.join(etc_folder, f'{fname}_{output_suffix}.pdf')
    new_doc.save(output_path)
    new_doc.close()
    doc.close()
    print(f"Extracted pages from {start_page} to end from {pdf_path} to {output_path}")

def process_all_pdfs(input_folder, output_folder, not_found_folder, etc_folder, output_suffix_1, output_suffix_2, keyword):
    pdf_files = glob.glob(os.path.join(input_folder, '*.pdf'))
    for pdf_file in pdf_files:
        found_page = find_keyword_end_page(pdf_file, keyword)
        if found_page:
            extract_pages_to_new_pdf(pdf_file, found_page, output_folder, output_suffix_1)
            extract_remaining_pages_to_new_pdf(pdf_file, found_page, etc_folder, output_suffix_2)
        else:
            # 키워드가 발견되지 않은 경우, 파일을 not_found_folder로 복사
            shutil.copy(pdf_file, os.path.join(not_found_folder, os.path.basename(pdf_file)))
            print(f"Keyword '{keyword}' not found in {pdf_file}, copied to {not_found_folder}")

# 경로와 키워드 설정
input_folder = r"C:\Users\user\Desktop\sample"  # 원본 PDF 파일이 있는 폴더 경로
output_folder = r"C:\Users\user\Desktop\extract"  # 이력서 파일을 저장할 폴더 경로
etc_folder =  r"C:\Users\user\Desktop\etc" # 충원요청서 체크리스트 파일을 저장할 폴더 경로
not_found_folder = r"C:\Users\user\Desktop\not_found" # 키워드 발견되지 않은 파일을 저장할 폴더 경로 (얘는 아마 수작업 필요)
output_suffix_1 = '이력서'  # 출력 파일 접두사1
output_suffix_2 = '충원요청서 체크리스트' # 출력 파일 접두사2
keyword = '충원요청서'  # 검색할 키워드
process_all_pdfs(input_folder, output_folder, not_found_folder, etc_folder, output_suffix_1, output_suffix_2, keyword)

Keyword '충원요청서' not found in C:\Users\user\Desktop\sample\결제서류_출석부__이력서.pdf, copied to C:\Users\user\Desktop\not_found
Extracted pages up to 16 from C:\Users\user\Desktop\sample\김다희(02514).pdf to C:\Users\user\Desktop\extract\김다희(02514)_이력서.pdf
Extracted pages from 16 to end from C:\Users\user\Desktop\sample\김다희(02514).pdf to C:\Users\user\Desktop\etc\김다희(02514)_충원요청서 체크리스트.pdf
Keyword '충원요청서' not found in C:\Users\user\Desktop\sample\김예리(01972).pdf, copied to C:\Users\user\Desktop\not_found
